# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import re

browser = webdriver.Chrome(executable_path = "../chromedriver")

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

year = "2018"

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value("11")

selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value(year)

browser.find_element_by_id("ctl05_btnQuery").click()

In [2]:
html_source = browser.page_source

soup = BeautifulSoup(html_source, "html.parser")
table = soup.find("table", class_ = "TABLE_G")

d = {}

type_string = "SO2"

for tr_node in table.find_all("tr"):
    if (len(tr_node.find_all("th")) > 0):
        continue
        
    for td_node in tr_node.find_all("td"):
        if (len(td_node.attrs) > 0):
            if (td_node.get("style") != None) :
                if (td_node.text == type_string):
                    d[type_string] = {}
                    
                    continue
                
            else:
                continue
        
        if (re.compile(f"{year}/0[1-8]").match(td_node.text)):
            next_td_node = td_node.findNext("td")
            
            d[type_string][td_node.text] = next_td_node.text
            
browser.quit()

In [3]:
import pandas as pd

pd.DataFrame(d)

,SO2
2018/01,10.77
2018/02,14.95
2018/03,13.56
2018/04,12.64
2018/05,12.94
2018/06,11.38
2018/07,8.92
2018/08,12.08


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import re

browser = webdriver.Chrome(executable_path = "../chromedriver")

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

year = "2018"

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value("11")

selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value(year)

browser.find_element_by_id("ctl05_btnQuery").click()

In [5]:
html_source = browser.page_source

soup = BeautifulSoup(html_source, "html.parser")
table = soup.find("table", class_ = "TABLE_G")

d = {}

type_string = ""

for tr_node in table.find_all("tr"):
    if (len(tr_node.find_all("th")) > 0):
        continue
        
    for td_node in tr_node.find_all("td"):
        if (len(td_node.attrs) > 0):
            if (td_node.get("style") != None) :
                if (re.compile("[CS]O\d*").match(td_node.text)):
                    type_string = td_node.text
                    d[type_string] = {}
                    
                    continue
                
            else:
                continue
        
        if (re.compile(f"{year}/0[1-8]").match(td_node.text)):
            next_td_node = td_node.findNext("td")
            
            d[type_string][td_node.text] = next_td_node.text
            
browser.quit()

In [6]:
import pandas as pd

pd.DataFrame(d)

,SO2,CO
2018/01,1.80,10.77
2018/02,1.90,14.95
2018/03,2.20,13.56
2018/04,2.30,12.64
2018/05,3.10,12.94
2018/06,2.70,11.38
2018/07,2.20,8.92
2018/08,2.40,12.08
